<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 警告。此文件不是BOOTCAMP的一部分
# 教学步骤。这是一个中至高级的例子
# 你将学到的东西。如果你是为了学习Chisel而参加BOOTCAMP，请不要从这里开始
# 请从[介绍Scala](1_intro_to_scala.ipynb)开始

# Chisel演示
**接下来：[介绍Scala](1_intro_to_scala.ipynb)**

欢迎！也许你是一个对"Chisel"这个名字感兴趣的学生，或者你是一位经验丰富的硬件设计专家，受到经理的委托，要探索Chisel作为新的HDL替代方案。无论哪种方式，如果你是Chisel的新手，你想尽快弄清楚到底发生了什么。别再犹豫了——让我们看看Chisel有什么特色！

## 设置
在我们开始之前，我们需要下载并导入演示所需的依赖项。

**请通过在键盘上按SHIFT+ENTER或在菜单中点击Run按钮来运行以下两个单元格块**。

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

## 硬件生成器：用于RTL的类型安全元编程

所有硬件描述语言都支持编写RTL设计的单个实例 - Chisel也不例外。
事实上，大多数Verilog/VHDL数字逻辑设计可以直接转录成Chisel！
虽然Chisel提供了我们将要学到的其他令人惊叹的功能，但我们要强调的是，切换到Chisel的用户将保留与任何其他硬件语言一样的对设计的精确控制程度。

看下面这个以FIR滤波器风格实现的3点移动平均值的例子。

<img src="images/demo_fir_filter.svg" width="512" />

Chisel提供类似的基本原语，可合成Verilog，并且*可以*像Verilog一样使用！运行下一个单元格以声明我们的Chisel模块。

In [ ]:
// 以FIR滤波器风格实现的3点移动平均值
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in) // 创建一个寄存器，其输入连接到参数io.in
  val z2 = RegNext(z1)    // 创建一个寄存器，其输入连接到参数z1

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U) // `1.U`是具有值1的无符号文字
}

在定义了`class MovingAverage3`之后，让我们实例化它并查看其结构：

In [ ]:
// 与之前相同的3点移动平均滤波器
visualize(() => new MovingAverage3(8))

在这个Chisel实例的可视化中，左侧是输入，右侧是z1和z2寄存器。这两个寄存器和io_in都分别与它们的系数相乘，然后逐个相加。`tail`和`bits`元素用于防止加法增长。

你可能会问：“嗯，这些在Verilog中可以做，但我为什么要使用Chisel呢？”

我们很高兴你问！Chisel的真正力量来自于能够创建**生成器，而不是实例**。假设我们不仅仅想要一个`MovingAverage3`模块，而是想要创建一个通用的`FIRFilter`模块，该模块由一系列系数参数化。

下面我们已经重写了`MovingAverage3`以接受系数序列。系数的数量将决定滤波器的大小。

In [ ]:
// 由卷积系数参数化的广义FIR滤波器
class FirFilter(bitWidth: Int, coeffs: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt())
  })
  // 创建串入并行出移位寄存器
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // 进行乘法
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i))

  // 求和乘积
  io.out := products.reduce(_ +& _)
}

现在，通过在实例化期间更改我们的`coeffs`参数，我们的`FIRFilter`模块可以用于实例化无数不同的硬件模块！下面我们创建了`FIRFiler`的三个不同实例。

In [ ]:
// 与之前相同的3点移动平均滤波器
visualize(() => new FirFilter(8, Seq(1.U, 1.U, 1.U)))

In [ ]:
// 作为FIR滤波器的1周期延迟
visualize(() => new FirFilter(8, Seq(0.U, 1.U)))

In [ ]:
// 具有三角脉冲响应的5点FIR滤波器
visualize(() => new FirFilter(8, Seq(1.U, 2.U, 3.U, 2.U, 1.U)))

没有这种强大的参数化，我们可能需要更多的模块定义，很可能是每个这些FIR滤波器的一个。理想情况下，我们希望我们的生成器具有（1）可组合性，（2）强大性，（3）对生成的设计进行精细控制的能力。

Chisel的好处在于你如何使用它，而不是语言本身。
如果你决定写实例而不是生成器，你将看到Chisel相对于Verilog的优势较少。
但是如果你花时间学习如何编写生成器，那么Chisel的威力将变得明显，你会意识到再也无法回到编写Verilog的方式。
学习编写生成器是困难的，但我们希望本教程能为您成为更好的硬件设计师、程序员和思考者铺平道路！

---
# 完成！

[返回顶部。](#top)